In [ ]:
import os
from langchain.llms import HuggingFaceHub

llm = HuggingFaceHub(
    huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
    repo_id="tiiuae/falcon-7b-instruct",
    model_kwargs={"temperature": 0.9, "max_new_tokens": 100}
)

result = llm("write a python program for sum")
print(result)

In [12]:
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path='codebasics_faqs.csv', source_column='prompt', encoding='ISO-8859-1')
data = loader.load()

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
import os
from langchain.vectorstores import FAISS
instructor_embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-large-en",
    query_instruction="Represent the query for retrieval:",
    model_kwargs={"token": os.getenv("HUGGINGFACEHUB_API_TOKEN")}
)



In [20]:
vectordb = FAISS.from_documents(documents=data, embedding=instructor_embeddings)

In [ ]:
retriever = vectordb.as_retriever()
rdocs = retriever.get_relevant_documents("how long is this course valid")


[Document(id='2e23e207-e5c4-4825-8c6b-0c927e49244d', metadata={'source': 'Once purchased, is this course available for lifetime access?', 'row': 22}, page_content='prompt: Once purchased, is this course available for lifetime access?\nresponse: Yes'),
 Document(id='4bc619da-f647-47f6-b550-8914f33989b9', metadata={'source': '\nI don\x92t have a laptop, can I take this course?', 'row': 25}, page_content='prompt: I don\x92t have a laptop, can I take this course?\nresponse: We recommend learning by doing and therefore you need to have a laptop or a PC (at least 4 GB ram).'),
 Document(id='b8dcc710-8a4a-4cfb-8938-516519f2561a', metadata={'source': 'What are the things I need to know before starting this course?', 'row': 17}, page_content='prompt: What are the things I need to know before starting this course?\nresponse: This course is for absolute beginners hence you do not need any specific skills other than basic familiarity with computers'),
 Document(id='0eae8da6-435a-4b6d-9d33-de8faaf8

In [28]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

In [29]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)


In [30]:
chain('Do you provide job assistance and also do you provide job gurantee?')

/var/folders/xk/rz9dwfbd6dd51vm92m5z9hd80000gn/T/ipykernel_85748/2066545439.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain('Do you provide job assistance and also do you provide job gurantee?')
/Users/adwivedi/Documents/Python/Langchain/venv/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


{'query': 'Do you provide job assistance and also do you provide job gurantee?',
 'result': 'Given the following context and a question, generate an answer based on this context only.\nIn the answer try to provide as much text as possible from "response" section in the source document context without making much changes.\nIf the answer is not found in the context, kindly state "I don\'t know." Don\'t try to make up an answer.\n\nCONTEXT: prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.\n\nprompt: Will this course guarantee me a job?\nresponse: We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gi

In [31]:
chain("Do you guys provide internship and also do you offer EMI payments?")

/Users/adwivedi/Documents/Python/Langchain/venv/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


{'query': 'Do you guys provide internship and also do you offer EMI payments?',
 'result': 'Given the following context and a question, generate an answer based on this context only.\nIn the answer try to provide as much text as possible from "response" section in the source document context without making much changes.\nIf the answer is not found in the context, kindly state "I don\'t know." Don\'t try to make up an answer.\n\nCONTEXT: prompt: Do you provide any virtual internship?\nresponse: Yes\n\nprompt: Do we have an EMI option?\nresponse: No\n\nprompt: I\x92m not sure if this bootcamp is good enough for me to invest some \nmoney. What can I do?\nresponse: We got you covered. Go ahead and watch our youtube videos if you like them and want to learn further then this bootcamp is the perfect extension.\n\nprompt: Can I add this course to my resume?\nresponse: Yes. Absolutely you can mention the AtliQ Hardware project experience in your resume with the relevant skills that you will le